<h1 align="center">Деревья решений</h1>

In [1]:
# подключение библиотек
import numpy as np

from sklearn import datasets
from sklearn import model_selection
from sklearn.metrics import r2_score

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

import random

1. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [2]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узл

In [3]:
# класс листа для задачи регрессии
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        prediction = np.mean(self.labels)        
        return prediction      

In [4]:
X, y = datasets.make_regression(n_samples=1000, n_features=15, random_state=42)

# Разобьем выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                                                    test_size = 0.25,
                                                                    random_state = 1)

In [5]:
# Расчет качества

def quality(left_labels, right_labels, current_quality):
    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_quality - p * criterion(left_labels) - (1 - p) * criterion(right_labels)

In [6]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [7]:
# Расчет критерия Джини

def criterion(labels):
     
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    for label in classes:
        c_criterion = np.mean((labels - np.mean(labels))**2) 
    return c_criterion

In [8]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5
    current_criterion = criterion(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_criterion)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [9]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, deep_tree = 5, count = 0):
    
    quality, t, index = find_best_split(data, labels)
   
    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)
    
    # в бинарном дереве нет смысла считать листья так как количество листьев будет 2**n, где n - глубина дерева
    # соответственно мы можем ограничить глубину:
    if count >= deep_tree:
        return Leaf(data, labels)
    
        
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    
    # инкрементируем счетчик глубины дерева
    count += 1
    
    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, deep_tree = deep_tree, count=count)
    false_branch = build_tree(false_data, false_labels, deep_tree = deep_tree, count=count)
    
   
    
    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [10]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [11]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [12]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index))
    print(spacing + 'Порог', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на отрицательном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [13]:
def computing(deep_tree = 3):
    # Построим дерево по обучающей выборке
    my_tree = build_tree(X_train, y_train, deep_tree = deep_tree)
    # Напечатаем ход нашего дерева
    print_tree(my_tree)
    # Получим ответы для обучающей выборки 
    train_answers = predict(X_train, my_tree)
    # И получим ответы для тестовой выборки
    answers = predict(X_test, my_tree)
    # Точность на обучающей выборке
    train_accuracy = r2_score(y_train, train_answers)
    print(f'Точность на обучающей выборке: {train_accuracy}')
    # Точность на тестовой выборке
    test_accuracy = r2_score(y_test, answers)
    print(f'Точность на тестовой выборке: {test_accuracy}')

In [14]:
computing() # дисперсия

Индекс 5
Порог -0.15350069556978385
--> True:
  Индекс 6
  Порог 0.46330151023423083
  --> True:
    Индекс 10
    Порог -0.6517041482182336
    --> True:
      Прогноз: -238.65033943130015
    --> False:
      Прогноз: -89.09338315777826
  --> False:
    Индекс 10
    Порог 0.019458255518883118
    --> True:
      Прогноз: -55.094430334890255
    --> False:
      Прогноз: 93.05661410966692
--> False:
  Индекс 6
  Порог -0.3386690538837576
  --> True:
    Индекс 10
    Порог 0.034271814200222225
    --> True:
      Прогноз: -94.42798263317393
    --> False:
      Прогноз: 57.79855727526092
  --> False:
    Индекс 10
    Порог 0.5809995399616921
    --> True:
      Прогноз: 78.05151680482207
    --> False:
      Прогноз: 209.42488977164697
Точность на обучающей выборке: 0.5357776496349808
Точность на тестовой выборке: 0.47877952599537865


In [15]:
computing(5)

Индекс 5
Порог -0.15350069556978385
--> True:
  Индекс 6
  Порог 0.46330151023423083
  --> True:
    Индекс 10
    Порог -0.6517041482182336
    --> True:
      Индекс 10
      Порог -1.4511756900515589
      --> True:
        Индекс 6
        Порог -0.599355705052997
        --> True:
          Прогноз: -450.87824312315337
        --> False:
          Прогноз: -257.09063255525444
      --> False:
        Индекс 2
        Порог -1.132784951267745
        --> True:
          Прогноз: -337.7488837531851
        --> False:
          Прогноз: -171.09176651493516
    --> False:
      Индекс 10
      Порог 1.0048216917579127
      --> True:
        Индекс 7
        Порог 0.012814304566494933
        --> True:
          Прогноз: -169.02337015269146
        --> False:
          Прогноз: -70.12618601042011
      --> False:
        Индекс 7
        Порог -0.2717300398697084
        --> True:
          Прогноз: -53.827748721430545
        --> False:
          Прогноз: 86.98907246511759
  --> Fals